# Model training and evaluation

This notebook includes the model training and evaluation for DeepLabv3 with ResNet101 and MobileNetV3 backbone layers.

In [ ]:
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torchvision import models

from utils import (
    get_device, train_test_split, batch_data, files_to_tensors,
    pixel_accuracy, iou_score, dice_score, f1_score
)

## Importing dataset

Data we use consist of POEM images and a corresponding 2D tensor comprising of the each pixel's label. The images are imported from a local directory and the tensors are store in a pickled dictionary containing the image file name as keys and tensors as values.

In [ ]:
im_dir = "/Users/naman/Workspace/Data/BM5020-POEM/Snapshots"
pkl_path = f"{im_dir}/annotations.pkl"
# pkl_path = "segmented_images.pkl"

classes = ["Background", "Muscle layer", "Mucosal layer", "Electrode"]

with open(pkl_path, "rb") as file:
    data_dict = pickle.load(file)

print(
    f"Size of data: {len(data_dict)}\n"
    f"Classes in data: {classes}"
)

## Data preprocessing

Here 

In [ ]:
all_files = list(data_dict.keys())

train_ratio = 0.8
shuffle = True
train_files, test_files = train_test_split(all_files, train_ratio, shuffle)

print(
    f"Train size: {len(train_files)}\n"
    f"Test size: {len(test_files)}"
)

In [ ]:
batch_size = 8
batched_train = batch_data(train_files, batch_size)

print(f"Number of batches: {len(batched_train)}")

## Loading the models

We load the models with classifier head as DeepLabv3 with 2 different backbones: Resnet101 and MobileNetV3. The classifier head in our model is used to classify each pixel and the backbone layer is used for feature extraction. Backbone layers are initialised with pretrained weights whereas the classifier head is initialised with random weights and 4 number of classes.

In [ ]:
device = get_device()

In [ ]:
num_classes = len(classes)

deeplabv3_weights = models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1
resnet101_weights = models.ResNet101_Weights.IMAGENET1K_V2
mobilenetv3_weights = models.mobilenet.MobileNet_V3_Large_Weights.IMAGENET1K_V2

deeplabv3_resnet101 = models.segmentation.deeplabv3.deeplabv3_resnet101(
    num_classes=num_classes, weights_backbone=resnet101_weights
).to(device)
deeplabv3_mobilenetv3 = models.segmentation.deeplabv3.deeplabv3_mobilenet_v3_large(
    num_classes=num_classes, weights_backbone=mobilenetv3_weights
).to(device)

In [ ]:
deeplabv3_resnet101

In [ ]:
deeplabv3_mobilenetv3

In [ ]:
# Choose the model to train
model = deeplabv3_mobilenetv3

## Model training and analysis

### Loading the optimizer and scheduler

We use the Adam optimizer with exponential scheduling. $\gamma$ in our scheduler is the factor multiplied after each step of the scheduler (which is taken after every epoch). Hence, the learning rate at $i\text{th}$ epoch will be $\text{initial\_lr} * \gamma^i$.

In [ ]:
initial_lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)

gamma = 0.8
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

### Training loop

In [ ]:
epochs = 40
batches = len(batched_train)

loss_list = []
accuracy_list = []
iou_list = [[] for _ in range(num_classes)]
dice_list = [[] for _ in range(num_classes)]
f1_list = [[] for _ in range(num_classes)]
lr_vals = []

for epoch in range(epochs):

    epoch_loss = 0
    epoch_accuracy = 0
    epoch_iou_list = [0] * num_classes
    epoch_dice_list = [0] * num_classes
    epoch_f1_list = [0] * num_classes

    for batch in batched_train:

        inputs, labels = files_to_tensors(batch, im_dir, data_dict)
        inputs = inputs.to(device)
        labels = labels.to(device)

        logits = model(inputs)["out"]

        loss = F.cross_entropy(logits, labels)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

        predictions = logits.argmax(dim=1).cpu()
        labels = labels.cpu()
        epoch_accuracy += pixel_accuracy(predictions, labels)

        for i in range(num_classes):
            epoch_iou_list[i] += iou_score(predictions == i, labels == i)
            epoch_dice_list[i] += dice_score(predictions == i, labels == i)
            epoch_f1_list[i] += f1_score(predictions == i, labels == i)

    lr_val = optimizer.state_dict()["param_groups"][0]["lr"]
    lr_vals.append(lr_val)
    scheduler.step()

    print(f"Epoch {epoch + 1}/{epochs} average loss: {epoch_loss / batches}")

    loss_list.append(epoch_loss / batches)
    accuracy_list.append(epoch_accuracy / batches)
    for i in range(num_classes):
        iou_list[i].append(epoch_iou_list[i] / batches)
        dice_list[i].append(epoch_dice_list[i] / batches)
        f1_list[i].append(epoch_f1_list[i] / batches)

### Plots and analysis

In [ ]:
# Plot learning rate schedule
plt.plot(lr_vals)
plt.xlabel("Epochs")
plt.title("Learning rate schedule")
plt.show()

In [ ]:
# Plot loss
plt.plot(loss_list)
plt.xlabel("Epochs")
plt.title("Training loss")
plt.show()

In [ ]:
# Plot pixel accuracy
plt.plot(accuracy_list)
plt.xlabel("Epochs")
plt.title("Training Pixel accuracy")
plt.show()

In [ ]:
# Plot IoU scores
for i, class_ in enumerate(classes):
    plt.plot(iou_list[i], label=f"Class {class_}")
plt.xlabel("Epochs")
plt.title(f"Training IoU score")
plt.legend()
plt.show()

In [ ]:
# Plot Dice scores
for i, class_ in enumerate(classes):
    plt.plot(dice_list[i], label=f"Class {class_}")
plt.xlabel("Epochs")
plt.title(f"Training Dice score")
plt.legend()
plt.show()

In [ ]:
# Plot F1 scores
for i, class_ in enumerate(classes):
    plt.plot(f1_list[i], label=f"Class {class_}")
plt.xlabel("Epochs")
plt.title(f"Training F1 score")
plt.legend()
plt.show()

## Model evaluation

In [ ]:
test_inputs, test_labels = files_to_tensors(test_files, im_dir, data_dict)
test_inputs = test_inputs.to(device)
test_labels = test_labels.to(device)

with torch.no_grad():
    logits = model(test_inputs)["out"]

test_labels = test_labels.cpu()
predictions = logits.argmax(dim=1).cpu()

In [ ]:
print(
    f"Test loss: {F.cross_entropy(logits, test_labels.to(device))}\n"
    f"Pixel accuracy on test set: {pixel_accuracy(predictions, test_labels)}\n"
)
for i, class_ in enumerate(classes):
    print(
        f"Class {class_} IoU score: {iou_score(predictions == i, test_labels == i)}\n"
        f"Class {class_} Dice score: {dice_score(predictions == i, test_labels == i)}\n"
        f"Class {class_} F1 score: {f1_score(predictions == i, test_labels == i)}\n"
    )